<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.3 
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerus than some, and does not have waiting period for approval of new developers (as of 18 August 2018).

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)


- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access 
- fill in the form fields at the bottom 
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"
 
- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [3]:
my_user_agent = 'johan_iod'   # your user Agent string goes in here
my_client_id = 'jYtdH12oUKdlxw'   # your Client ID string goes in here
my_client_secret = 's029p3n2wFwuiNrmv3eZQbIeQpW8gA'   # your Secret string goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [4]:
pwd()  # make sure your working directory is where the file is

'/Users/JohanNg/Institute of Data/Institute-of-Data/Lab 2'

In [5]:
# path_auth = 'auth_reddit.json'
# auth = json.loads(open(path_auth).read())
# pp = pprint.PrettyPrinter(indent=4)
# # For debugging only:
# #pp.pprint(auth)

# my_user_agent = auth['my_user_agent']
# my_client_id = auth['my_client_id']
# my_client_secret = auth['my_client_secret']

Security considerations: 
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file 
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [6]:
reddit = praw.Reddit(client_id = my_client_id, 
                     client_secret = my_client_secret, 
                     user_agent = my_user_agent)

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [7]:
reddit.subreddit

Consult the PRAW and Reddit API documentation. Print a few of the response members below:

In [8]:
print(reddit.comment)

<bound method Reddit.comment of <praw.reddit.Reddit object at 0x7fbba9df0490>>


Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic. 

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [9]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Ask Anything Monday - Weekly Thread
ELI5: bitwise operators
Struggling With Masters Due To Depression
What is the definition of "immutability"?
flask migrate error
How to synthesize tts with custom voices and solve dependency issues?
How to run RSS Parser periodically to grab new items? (Discord Bot)
Created TicTacToe - Looking for Feedback
Entering in Python world coming from other technologies.
How to change the text of label more than one time, Tkinter


Now retrieve 10 authors:

In [10]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author)

AutoModerator
dasfreak
ALostKashmiri
DuePresentation3
foollooweer
195monke
RainingRythm
AverageNobody_2
ReggaeShark09
himanshu_013


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

In [11]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)
    print(submission.author)
    print(submission.url)

Ask Anything Monday - Weekly Thread
AutoModerator
https://www.reddit.com/r/learnpython/comments/kletwf/ask_anything_monday_weekly_thread/
ELI5: bitwise operators
dasfreak
https://www.reddit.com/r/learnpython/comments/lll14x/eli5_bitwise_operators/
Struggling With Masters Due To Depression
ALostKashmiri
https://www.reddit.com/r/learnpython/comments/ll40a9/struggling_with_masters_due_to_depression/
What is the definition of "immutability"?
DuePresentation3
https://www.reddit.com/r/learnpython/comments/llot3f/what_is_the_definition_of_immutability/
flask migrate error
foollooweer
https://www.reddit.com/r/learnpython/comments/llqe35/flask_migrate_error/
How to synthesize tts with custom voices and solve dependency issues?
195monke
https://www.reddit.com/r/learnpython/comments/llqd6h/how_to_synthesize_tts_with_custom_voices_and/
How to run RSS Parser periodically to grab new items? (Discord Bot)
RainingRythm
https://www.reddit.com/r/learnpython/comments/llllqz/how_to_run_rss_parser_periodic

Why doesn't the next cell produce output?

In [12]:
for submission in submissions:
    print(submission.comments)

NameError: name 'submissions' is not defined

Print two comments associated with each of these submissions:

In [ ]:
submissions = reddit.subreddit('learnpython').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

Referring to the API documentation, explore the submissions object and print some interesting data:

In [13]:
submit = reddit.subreddit('depression').top(limit=10)

for submission in submit:
    print(submission.title)
    print(submission.subreddit)# Output: the submission's title
#     print(submission.score)  # Output: the submission's score
#     print(submission.id)     # Output: the submission's ID
#     print(submission.url)
    print(submission.num_comments)

Depression for me is a constant feeling of wanting to go home, but no matter where I am, I am never home. Even when I’m at my physical home.
depression
380
Depression is so much worse if you live with your parents.
depression
584
Shout out to the particular hell that is functional depression.
depression
350
A stranger just saved my life...
depression
171
I was going to kill myself tonight
depression
260
Does anyone ever feel okay for a while, then a wave of depression just hits you like a truck?
depression
367
The worst thing is realising no one is coming to save you and you have to rescue yourself with zero motivation to do so.
depression
282
Living with depression is like running a marathon with a broken leg and then having everyone run past you and say “Yeah, yeah, broken leg boo-hoo, we’re all tired.” And then run the same fucking race everyday. The cycle.
depression
140
I hate that people don’t understand that i don’t want to kill myself, I just don’t want to be alive anymore
depr

#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request: 

In [ ]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

>
>


>
>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



